In [1]:
# https://github.com/shoji9x9/CIFAR-10-By-small-ResNet/blob/master/ResNet-for-CIFAR-10-with-Keras.ipynb

In [2]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs.constants import MODELS_FOLDER

In [3]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
import libs.model_archs
import libs.utils
from libs.seeds import load_model_seeds
model_seeds = load_model_seeds()

In [5]:
# define dataset and model architecture
dataset = "cifar10"
# Select ResNet Version
resnet_version = 1
model_arch = "resnetA" + str(resnet_version)

# set training hyperparameters
batch_size = 128
n_epochs = 200

In [6]:
# global seed
seed = model_seeds[0]
tf.random.set_seed(seed)
np.random.seed(seed)

# prepare data
dataset_loader = getattr(libs.utils, 'prepare_'+dataset)
to_categorical=False
(x_train, y_train), (x_test, y_test) = dataset_loader(to_categorical)

In [7]:
# create model
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, Activation, MaxPool2D, GlobalAveragePooling2D, Add, Input, Flatten
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2


input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
n_classes = y_train.shape[1]

# n_channels = x_train.shape[3] # no. of channels (?)
# # Computed depth of model
# if resnet_version == 1:
# 	depth = n_channels * 6 + 2
# elif resnet_version == 2:
# 	depth = n_channels * 9 + 2

# model_generator = getattr(libs.model_archs, model_arch)
# model = model_generator(input_shape, depth, n_classes)

# https://github.com/shoji9x9/CIFAR-10-By-small-ResNet/blob/master/ResNet-for-CIFAR-10-with-Keras.ipynb
n = 9 # 56 layers
channels = [16, 32, 64]

inputs = Input(shape=input_shape)
x = Conv2D(channels[0], 
           kernel_size=(3, 3), 
           padding="same", 
           kernel_initializer="he_normal", 
           kernel_regularizer=l2(1e-4))(inputs)
x = BatchNormalization()(x)
x = Activation(tf.nn.relu)(x)

for c in channels:
    for i in range(n):
        subsampling = i == 0 and c > 16
        strides = (2, 2) if subsampling else (1, 1)
        y = Conv2D(c, 
                   kernel_size=(3, 3), 
                   padding="same", 
                   strides=strides, 
                   kernel_initializer="he_normal", 
                   kernel_regularizer=l2(1e-4))(x)
        y = BatchNormalization()(y)
        y = Activation(tf.nn.relu)(y)
        y = Conv2D(c, 
                   kernel_size=(3, 3), 
                   padding="same", 
                   kernel_initializer="he_normal", 
                   kernel_regularizer=l2(1e-4))(y)
        y = BatchNormalization()(y)
        if subsampling:
            x = Conv2D(c, 
                       kernel_size=(1, 1), 
                       strides=(2, 2), 
                       padding="same", 
                       kernel_initializer="he_normal", 
                       kernel_regularizer=l2(1e-4))(x)
        x = Add()([x, y])
        x = Activation(tf.nn.relu)(x)

x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
outputs = Dense(10, 
                activation=tf.nn.softmax, 
                kernel_initializer="he_normal")(x)

model = Model(inputs=inputs, outputs=outputs)

model._name = "resnet" + str(6 * n + 2)

In [8]:
# compile model
from tensorflow.keras.optimizers import SGD
# model.compile(loss ='categorical_crossentropy',
#               optimizer ='adam',
#               metrics =['accuracy'])
lr = 0.1
optimizer = SGD(learning_rate=lr, momentum=0.9)
model.compile(optimizer=optimizer, 
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])


# train model
model.fit(x_train, 
          y_train, 
          batch_size=batch_size, 
          epochs=n_epochs,
          validation_split=0.2)

Epoch 1/200
313/313 [==============================] - 69s 137ms/step - loss: 3.1882 - accuracy: 0.1886 - val_loss: 2.8692 - val_accuracy: 0.2195
Epoch 2/200
313/313 [==============================] - 37s 118ms/step - loss: 2.5589 - accuracy: 0.3111 - val_loss: 2.6552 - val_accuracy: 0.2717
Epoch 3/200
313/313 [==============================] - 37s 119ms/step - loss: 2.2001 - accuracy: 0.4236 - val_loss: 2.2438 - val_accuracy: 0.4127
Epoch 4/200
313/313 [==============================] - 38s 123ms/step - loss: 1.9477 - accuracy: 0.4992 - val_loss: 1.9292 - val_accuracy: 0.4909
Epoch 5/200
313/313 [==============================] - 37s 119ms/step - loss: 1.7551 - accuracy: 0.5552 - val_loss: 1.9281 - val_accuracy: 0.4827
Epoch 6/200
313/313 [==============================] - 37s 117ms/step - loss: 1.5952 - accuracy: 0.6012 - val_loss: 2.0012 - val_accuracy: 0.4629
Epoch 7/200
313/313 [==============================] - 39s 125ms/step - loss: 1.4398 - accuracy: 0.6439 - val_loss: 1.6335 -

In [9]:
# evaluate model
score = model.evaluate(x_test, 
                       y_test, 
                       batch_size=batch_size)

print("Original Accuracy: ",score)

79/79 [==============================] - 3s 31ms/step - loss: 1.5790 - accuracy: 0.7451
Original Accuracy:  [1.5790451765060425, 0.7451000213623047]


In [10]:
# save model
model_type = dataset + "--" + model_arch
model_instance = model_type + "-" + str(seed)
model_filename = model_instance + ".h5"
model_subdir = pathlib.Path(MODELS_FOLDER / model.name)
pathlib.Path(model_subdir).mkdir(parents=True, exist_ok=True)
model_file = str(pathlib.Path(model_subdir/ model_filename))
model.save(model_file)

In [11]:
model.summary()

Model: "resnet56"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0